# 1. TensorFlow (tf.keras) Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0  # Normalize to [0, 1]
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)  # One-hot encode

# Choose model: VGG16 or VGG19
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
# base_model = VGG19(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom classification head
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

# Freeze base model layers for transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-4:]:  # Unfreeze last 4 layers
    layer.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 22s 21ms/step - accuracy: 0.3918 - loss: 1.7213 - val_accuracy: 0.5420 - val_loss: 1.2969
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.5386 - loss: 1.3174 - val_accuracy: 0.5731 - val_loss: 1.2210
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.5622 - loss: 1.2484 - val_accuracy: 0.5790 - val_loss: 1.1956
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.5756 - loss: 1.2150 - val_accuracy: 0.5884 - val_loss: 1.1690
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.5905 - loss: 1.1824 - val_accuracy: 0.5891 - val_loss: 1.1683
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.5880 - loss: 1.1676 - val_accuracy: 0.5983 - val_loss: 1.1388
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.5985 - loss: 1.1451 - val_accuracy: 0.6031 - val_lo

# 2. PyTorch Lightning Implementation

In [ ]:
!pip install pytorch-lightning==2.0.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 58.5 MB/s eta 0:00:00


In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch import nn, optim
import pytorch_lightning as pl
from torchvision.models import vgg16, vgg19

# Define DataModule for CIFAR-10
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, batch_size=64):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    def prepare_data(self):
        datasets.CIFAR10(root='./data', train=True, download=True)
        datasets.CIFAR10(root='./data', train=False, download=True)

    def setup(self, stage=None):
        self.train_set = datasets.CIFAR10(root='./data', train=True, transform=self.transform)
        self.test_set = datasets.CIFAR10(root='./data', train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)

# Define the LightningModule
class VGGTransferLearning(pl.LightningModule):
    def __init__(self, model_type="vgg16", num_classes=10):
        super(VGGTransferLearning, self).__init__()
        if model_type == "vgg16":
            self.base_model = vgg16(pretrained=True)
        elif model_type == "vgg19":
            self.base_model = vgg19(pretrained=True)

        # Replace classifier for CIFAR-10
        self.base_model.classifier[6] = nn.Linear(4096, num_classes)

        # Freeze feature extractor layers
        for param in self.base_model.features.parameters():
            param.requires_grad = False

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.base_model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = self.criterion(preds, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = self.criterion(preds, y)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.base_model.classifier.parameters(), lr=1e-4)

# Initialize data module and model
data_module = CIFAR10DataModule(batch_size=64)
model = VGGTransferLearning(model_type="vgg16", num_classes=10)

# Train the model
trainer = pl.Trainer(max_epochs=10, accelerator="gpu", devices=1 if torch.cuda.is_available() else 0)
trainer.fit(model, data_module)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


100%|██████████| 170M/170M [00:13<00:00, 12.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type             | Params
------------------------------------------------
0 | base_model | VGG              | 134 M 
1 | criterion  | CrossEntropyLoss | 0     
------------------------------------------------
119 M     Trainable params
14.7 M    Non-trainable params
134 M     Total params
537.206   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


# early stop and dropout

# 1. TensorFlow (tf.keras) Implementation
early stop and dropout

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0  # Normalize to [0, 1]
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)  # One-hot encode

# Choose model: VGG16 or VGG19
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
# base_model = VGG19(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom classification head with Dropout
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Add Dropout for regularization
output = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

# Freeze base model layers for transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with EarlyStopping
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=64, callbacks=[early_stopping])

# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-4:]:  # Unfreeze last 4 layers
    layer.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# Evaluate model
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
y_true_classes = y_test.argmax(axis=1)

# Print classification report (accuracy, precision, recall, F1-score)
print(classification_report(y_true_classes, y_pred_classes))


Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.3966 - loss: 1.7061 - val_accuracy: 0.5483 - val_loss: 1.2962
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.5372 - loss: 1.3181 - val_accuracy: 0.5753 - val_loss: 1.2233
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.5570 - loss: 1.2628 - val_accuracy: 0.5840 - val_loss: 1.1861
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.5725 - loss: 1.2141 - val_accuracy: 0.5913 - val_loss: 1.1702
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.5872 - loss: 1.1798 - val_accuracy: 0.5970 - val_loss: 1.1504
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.5929 - loss: 1.1653 - val_accuracy: 0.6016 - val_loss: 1.1380
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.6009 - loss: 1.1432 - val_accuracy: 0.6028 - val_loss: 1.1369
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.6088 - loss: 1.1118 - val

# 2. PyTorch Lightning Implementation

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch import nn, optim
import pytorch_lightning as pl
from torchvision.models import vgg16, vgg19
from sklearn.metrics import classification_report
import numpy as np

# Define DataModule for CIFAR-10
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, batch_size=64):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    def prepare_data(self):
        datasets.CIFAR10(root='./data', train=True, download=True)
        datasets.CIFAR10(root='./data', train=False, download=True)

    def setup(self, stage=None):
        self.train_set = datasets.CIFAR10(root='./data', train=True, transform=self.transform)
        self.test_set = datasets.CIFAR10(root='./data', train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)

# Define the LightningModule with Early Stopping and Dropout
class VGGTransferLearning(pl.LightningModule):
    def __init__(self, model_type="vgg16", num_classes=10):
        super(VGGTransferLearning, self).__init__()
        if model_type == "vgg16":
            self.base_model = vgg16(pretrained=True)
        elif model_type == "vgg19":
            self.base_model = vgg19(pretrained=True)

        # Replace classifier for CIFAR-10 with Dropout
        self.base_model.classifier[6] = nn.Sequential(
            nn.Linear(4096, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

        # Freeze feature extractor layers
        for param in self.base_model.features.parameters():
            param.requires_grad = False

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.base_model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = self.criterion(preds, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = self.criterion(preds, y)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.base_model.classifier.parameters(), lr=1e-4)

# Initialize data module and model
data_module = CIFAR10DataModule(batch_size=64)
model = VGGTransferLearning(model_type="vgg16", num_classes=10)

# Early Stopping callback
early_stop_callback = pl.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with EarlyStopping
trainer = pl.Trainer(max_epochs=20, gpus=1 if torch.cuda.is_available() else 0, callbacks=[early_stop_callback])
trainer.fit(model, data_module)

# Evaluate model
y_pred = trainer.predict(model, data_module.val_dataloader())
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.array([y for _, y in data_module.test_set])

# Print classification report (accuracy, precision, recall, F1-score)
print(classification_report(y_true_classes, y_pred_classes))
